In [1]:
from collections import defaultdict, Counter
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import string
import time
import config as rs
import pickle as pcl
import os.path
from collections import OrderedDict

In [2]:
# Database works here.
rs.collect("http://rss.cnn.com/rss/cnn_topstories.rss", 10, filename="news.pickle")

downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/4YFwJMnmxVI/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/pGIa9Im6FdU/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/NVj0Ls3EutU/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/K-WTe1Qto9s/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/7f4Yf2JdhRw/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/E55VK05GPD4/index.html
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/VKwZXVM56n8/senate-vote-skinny-obamacare-repeal-timeline-ekr-orig-vstan.cnn
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/G909gdddVAk/mccain-no-vote-skinny-repeal-mxb-lon-orig.cnn
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/1RdzUO-0u3I/senate-vote-skinny-repeal-bill-obamacare-schumer-on-mccain-sot.cnn
downloading: http://rss.cnn.com/~r/rss/cnn_topstories/~3/SneKysBn2O8/index.html


In [22]:
news = rs.load()
print(type(news.keys()))
print(type(news.values()))
len(news)

<class 'odict_keys'>
<class 'odict_values'>


10

In [20]:
string = rs.get_text("http://rss.cnn.com/~r/rss/cnn_topstories/~3/4YFwJMnmxVI/index.html")
type(string)

str

In [21]:
rs.tokenize(string)

['mccain',
 'who',
 'had',
 'voted',
 'for',
 'a',
 'motion',
 'to',
 'proceed',
 'to',
 'the',
 'bill',
 'tuesday',
 'after',
 'returning',
 'to',
 'washington',
 'following',
 'surgery',
 'for',
 'a',
 'brain',
 'tumor',
 'held',
 'out',
 'all',
 'day',
 'including',
 'in',
 'a',
 'news',
 'conference',
 'where',
 'he',
 'criticized',
 'the',
 'partisan',
 'process',
 'that',
 'led',
 'to',
 'the',
 'after-midnight',
 'vote',
 'his',
 'surprise',
 'decision',
 'came',
 'after',
 'a',
 'prolonged',
 'drama',
 'on',
 'the',
 'senate',
 'floor',
 'multiple',
 'republican',
 'colleagues',
 'including',
 'vice',
 'president',
 'mike',
 'pence',
 'engaged',
 'in',
 'animated',
 'conversations',
 'with',
 'the',
 'arizona',
 'senator',
 'who',
 'has',
 'long',
 'cherished',
 'his',
 'reputation',
 'as',
 'a',
 'maverick',
 'at',
 'one',
 'point',
 'before',
 'the',
 'final',
 'vote',
 'trump',
 'called',
 'pence',
 'who',
 'handed',
 'the',
 'phone',
 'to',
 'mccain',
 'a',
 'source',
 'bri